In [ ]:
!pip install caikit-nlp-client

In [ ]:
from caikit_nlp_client import GrpcClient, HttpClient
import os

In [ ]:
MODEL_ID = os.environ["MODEL_ID"]
HTTP_HOST = os.environ["HTTP_HOST"]
GRPC_HOST = os.environ["GRPC_HOST"]
PORT = os.environ["PORT"]
QUERY_TEXT = os.environ["QUERY_TEXT"] # text = "At what temperature does water boil?"
EXPECTED_ANSWER = os.environ["EXPECTED_ANSWER"]

# HTTP

## skip ssl validation

In [ ]:
%%capture --no-display
http_client = HttpClient(f"{HTTP_HOST}:{PORT}", verify=False)
generated_text = http_client.generate_text(MODEL_ID, QUERY_TEXT)

In [ ]:
print(generated_text)
assert generated_text.strip() == EXPECTED_ANSWER.strip()

## with TLS cert

In [ ]:
http_client = HttpClient(f"{HTTP_HOST}:{PORT}", ca_cert_path="openshift_ca_istio_knative.crt")
generated_text = http_client.generate_text(MODEL_ID, QUERY_TEXT)
print(generated_text)
assert generated_text.strip() == EXPECTED_ANSWER.strip()

## with mTLS

In [ ]:
http_client = HttpClient(f"{HTTP_HOST}:{PORT}",
                         ca_cert_path="openshift_ca_istio_knative.crt",
                         client_cert_path="public.crt",
                         client_key_path="private.key")
generated_text = http_client.generate_text(MODEL_ID, QUERY_TEXT)
print(generated_text)
assert generated_text.strip() == EXPECTED_ANSWER.strip()

# gRPC

## skip ssl validation

In [ ]:
%%capture --no-display
grpc_client = GrpcClient(GRPC_HOST, int(PORT), verify=False)

In [ ]:
generated_text = grpc_client.generate_text(MODEL_ID, QUERY_TEXT)
print(generated_text)
assert generated_text.strip() == EXPECTED_ANSWER.strip()

## with TLS cert

In [ ]:
with open("openshift_ca_istio_knative.crt", "rb") as fh:
    ca_cert = fh.read()
grpc_client = GrpcClient(GRPC_HOST, int(PORT), ca_cert=ca_cert)
generated_text = grpc_client.generate_text(MODEL_ID, QUERY_TEXT)
print(generated_text)
assert generated_text.strip() == EXPECTED_ANSWER.strip()

## with mTLS

In [ ]:
with open("openshift_ca_istio_knative.crt", "rb") as fh:
    ca_cert = fh.read()
with open("public.crt", "rb") as fh:
    client_cert = fh.read()
with open("private.key", "rb") as fh:
    client_key = fh.read()


grpc_client = GrpcClient(GRPC_HOST, int(PORT), ca_cert=ca_cert, client_cert=client_cert, client_key=client_key)
generated_text = grpc_client.generate_text(MODEL_ID, QUERY_TEXT)
print(generated_text)
assert generated_text.strip() == EXPECTED_ANSWER.strip()